#Dependencies and Setup#

In [ ]:
!pip install transformers
!pip install datasets transformers[SentencePiece]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'CS682/CS682-Project/Final'

import sys
sys.path.append('/content/drive/My Drive/{}/code'.format(FOLDERNAME))

%load_ext autoreload
%autoreload 2

%cd /content/drive/My\Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/17sOv4AaDBYiDb5d0-jvQZQFiRXJ7Ui5-/project-shared/Final


In [ ]:
from datasets import *
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, DataCollatorWithPadding, get_scheduler
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from solver import *
from models import *
from rmse import *
from utils import *
from log_control import *
from transformers import logging
from datetime import date
date = date.today()
model_lib = {}
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

#Single-Task | Base Model

The next 2 cells are meant to be run together

In [ ]:
#Classification Training Loops

# set_global_logging_level(logging.INFO)
set_global_logging_level(logging.ERROR, ["transformers", "nlp", "torch", "tensorflow", "tensorboard", "wandb", "datasets"])
# logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

experiment = 'single_base_5' #experiment type, also used to create output files
logging.set_verbosity_error()
checkpoints = ['bert-base-uncased','roberta-base', 'microsoft/deberta-base', 'albert-base-v2', 'nghuyong/ernie-2.0-en'] #Pre-Trained Model Checkpoints
tasks = ['is_humor', 'humor_rating', 'humor_controversy', 'offense_rating'] #Tasks
setup = 'single' #'multi' if multi-task model, otherwise 'single'
fc_dim = 256 # Hidden layer size of our classifier
has_decoder=False # We don't have an LSTM layer for this experiment]
batch_size=24 #input batch_size for our dataloader
epochs = 2
checkpoints = ['nghuyong/ernie-2.0-en']


df_train, df_val, df_test = get_split_dfs('cs682-humor-offense/data/train.csv')
raw_datasets = DatasetDict({'train':Dataset.from_pandas(df_train), 'validation':Dataset.from_pandas(df_val), 'test':Dataset.from_pandas(df_test)})
new_df_val=df_val.copy() # copy validation DataFrame to enter results and write to file

#Iterate for (#pre-trained models * #tasks) combinations
for checkpoint in checkpoints:
  for task in tasks:
    
    #initialize model, tokenizer and optimizer
    model = base_model(checkpoint=checkpoint,setup='single',task=task,has_decoder=has_decoder,fc_dim=fc_dim)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    optimizer = AdamW(model.parameters(), lr=3e-5)
    
    model.to(device)

    #get tokenized datasets for this particular task
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    tokenized_datasets = get_tokenized_datasets(raw_datasets, tokenizer,task)
    

    #create dataloaders from tokenized datasets
    train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=batch_size, collate_fn=data_collator)
    eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=batch_size, collate_fn=data_collator)

    print()
    print("Model: %s | Task:%s" % (checkpoint,task))
    #train and save model
    best_model = train_loop(model, optimizer, train_dataloader, eval_dataloader, epochs=epochs)
    # model.save_pretrained('/content/drive/My Drive/{}/saved_models/{}_tuned'.format(FOLDERNAME,checkpoint))


    #evaluate and save to CSV
    perfs, pred = eval_model(best_model, eval_dataloader)
    new_df_val['pred_'+checkpoint+'_'+task] = pred
    new_df_val.to_csv('cs682-humor-offense/output/'+experiment+'.csv')

    model_lib[(experiment,task, checkpoint)] = [best_model, perfs]
    df_summary = pd.DataFrame.from_dict(model_lib, orient='index')
    df_summary.to_csv('cs682-humor-offense/output/'+experiment+'_summary.csv')
    
    
    for perf in perfs:
      for _,key in enumerate(perf):
        print ("%s = %0.2f" % (key,perf[key]))
    print()


Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]


Model: nghuyong/ernie-2.0-en | Task:offense_rating
Training Loop...


  0%|          | 0/534 [00:00<?, ?it/s]


Started Epoch 1/2:
Iteration 1/267: | Loss = 0.64 | [{'rmse': 0.8993052874302474}]
Iteration 83/267: | Loss = 0.12 | [{'rmse': 0.6314665470157544}]
Iteration 166/267: | Loss = 0.20 | [{'rmse': 0.6154266812545586}]
Iteration 249/267: | Loss = 0.36 | [{'rmse': 0.6373774391990981}]
Iteration 267/267: | Loss = 0.36 | [{'rmse': 0.5809475019311126}]

Started Epoch 2/2:
Iteration 1/267: | Loss = 0.12 | [{'rmse': 0.5830951894845301}]
Iteration 83/267: | Loss = 0.10 | [{'rmse': 0.544288526426931}]
Iteration 166/267: | Loss = 0.07 | [{'rmse': 0.6294839156007086}]
Iteration 249/267: | Loss = 0.08 | [{'rmse': 0.552268050859363}]
Iteration 267/267: | Loss = 0.11 | [{'rmse': 0.5477225575051661}]
Best Performance: 0.544288526426931
rmse = 0.55



##Single-Task, Base Model, Ensemble##

In [ ]:
#Performance
# Single-Task, Base Model, Ensemble
res_single_base_ens = compute_ensemble('base_ensemble')

print("Performance: Single-Task | Base-Model | Ensemble")
print("************************************************")
print()
for key in res_single_base_ens:
  print(key)
  print(res_single_base_ens[key])
  print()


Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Performance: Single-Task | Base-Model | Ensemble
************************************************

is_humor
({'f1': 0.9631901840490799}, {'accuracy': 0.955})

humor_controversy
({'f1': 0.5476190476190476}, {'accuracy': 0.715})

humor_rating
{'rmse': 0.6214901447328026}

offense_rating
{'rmse': 0.532681893816563}



#Single-Task | Base Model + LSTM

The next 2 cells are meant to be run together

In [ ]:
#Classification Training Loops

# set_global_logging_level(logging.INFO)
set_global_logging_level(logging.ERROR, ["transformers", "nlp", "torch", "tensorflow", "tensorboard", "wandb", "datasets"])
# logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

experiment = 'single_lstm_5' #experiment type, also used to create output files
logging.set_verbosity_error()
checkpoints = ['bert-base-uncased','roberta-base', 'microsoft/deberta-base', 'albert-base-v2', 'nghuyong/ernie-2.0-en'] #Pre-Trained Model Checkpoints
tasks = ['is_humor', 'humor_rating', 'humor_controversy', 'offense_rating'] #Tasks
setup = 'single' #'multi' if multi-task model, otherwise 'single'
fc_dim = 256 # Hidden layer size of our classifier
lstm_dim = 256
has_decoder=True # We have an LSTM layer for this experiment
batch_size=32 #input batch_size for our dataloader
epochs = 2
checkpoints = ['nghuyong/ernie-2.0-en']

df_train, df_val, df_test = get_split_dfs('cs682-humor-offense/data/train.csv')
raw_datasets = DatasetDict({'train':Dataset.from_pandas(df_train), 'validation':Dataset.from_pandas(df_val), 'test':Dataset.from_pandas(df_test)})
new_df_val=df_val.copy() # copy validation DataFrame to enter results and write to file

#Iterate for (#pre-trained models * #tasks) combinations
for checkpoint in checkpoints:
  for task in tasks:
    
    #initialize model, tokenizer and optimizer
    # model = base_model(checkpoint=checkpoint,setup='single',task=task,has_decoder=has_decoder,fc_dim=fc_dim, lstm_dim=lstm_dim)
    # tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    optimizer = AdamW(model.parameters(), lr=3e-5)
    
    model.to(device)

    #get tokenized datasets for this particular task
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    tokenized_datasets = get_tokenized_datasets(raw_datasets, tokenizer,task)
    

    #create dataloaders from tokenized datasets
    train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=batch_size, collate_fn=data_collator)
    eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=batch_size, collate_fn=data_collator)
    break

    print()
    print("Model: %s | Task:%s" % (checkpoint,task))
    #train and save model
    best_model = train_loop(model, optimizer, train_dataloader, eval_dataloader, epochs=epochs)
    # model.save_pretrained('/content/drive/My Drive/{}/saved_models/{}_tuned'.format(FOLDERNAME,checkpoint))


    #evaluate and save to CSV
    perfs, pred = eval_model(best_model, eval_dataloader)
    new_df_val['pred_'+checkpoint+'_'+task] = pred
    new_df_val.to_csv('cs682-humor-offense/output/'+experiment+'.csv')

    model_lib[(experiment,task, checkpoint)] = [best_model, perfs]
    df_summary = pd.DataFrame.from_dict(model_lib, orient='index')
    df_summary.to_csv('cs682-humor-offense/output/'+experiment+'_summary.csv')
    
    
    for perf in perfs:
      for _,key in enumerate(perf):
        print ("%s = %0.2f" % (key,perf[key]))
    print()
  break

##Single-Task, Base Model + LSTM, Ensemble##

In [ ]:
#TEMPORARY CODE FOR ENSEMBLE COMPUTATION BEFORE PROJECT SUBMISSION
# Single-Task, Base Model, Ensemble
res_single_base_ens = compute_ensemble('lstm_ensemble')

print("Performance: Single-Task | Base-Model | Ensemble")
print("************************************************")
print()
for key in res_single_base_ens:
  print(key)
  print(res_single_base_ens[key])
  print()


Performance: Single-Task | Base-Model | Ensemble
************************************************

is_humor
({'f1': 0.9551934826883911}, {'accuracy': 0.945})

humor_controversy
({'f1': 0.5280000000000001}, {'accuracy': 0.705})

humor_rating
{'rmse': 0.6383572667401852}

offense_rating
{'rmse': 0.5689903338370521}



In [ ]:
#Performance
# Single-Task, Base Model, Ensemble
res_single_lstm_ens = compute_ensemble(experiment)

print("Performance: Single-Task | Base-LSTM | Ensemble")
print("************************************************")
print()
for key in res_single_lstm_ens:
  print(key)
  print(res_single_lstm_ens[key])
  print()
